In [1]:
%pip install pyreadstat pandas openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# python sav2xlsx.py /path/to/folder /path/to/output 
import argparse
import sys
import os
from pathlib import Path
from typing import Dict, List, Tuple
import pandas as pd

try:
    import pyreadstat
except ImportError as e:
    print("ERROR: pyreadstat is required. Install with:\n  pip install pyreadstat pandas openpyxl", file=sys.stderr)
    sys.exit(1)


def build_value_labels_df(value_labels: Dict[str, Dict]) -> pd.DataFrame:
    """
    Convert pyreadstat value labels dict into a tidy DataFrame with columns:
    variable, value, label
    """
    rows: List[Tuple[str, str, str]] = []
    for var, mapping in value_labels.items():
        # mapping is {code: label}
        for code, label in mapping.items():
            rows.append((var, code, label))
    if not rows:
        return pd.DataFrame(columns=["variable", "value", "label"])
    df = pd.DataFrame(rows, columns=["variable", "value", "label"])
    return df.sort_values(["variable", "value"]).reset_index(drop=True)


def build_variable_info_df(meta: "pyreadstat.metadata_container.MetadataContainer") -> pd.DataFrame:
    """
    Create a DataFrame summarizing variable metadata.
    """
    names = meta.column_names or []
    labels_map = meta.column_names_to_labels or {}
    measure_map = meta.measure_levels or {}
    formats_map = meta.formats or {}
    missing_ranges = meta.missing_ranges or {}
    missing_values = meta.missing_user_values or {}

    rows = []
    for name in names:
        rows.append({
            "name": name,
            "label": labels_map.get(name, ""),
            "measure_level": measure_map.get(name, ""),
            "format": formats_map.get(name, ""),
            "missing_values": str(missing_values.get(name, "")),
            "missing_ranges": str(missing_ranges.get(name, "")),
        })
    return pd.DataFrame(rows)


def convert_file(input_path: Path, output_dir: Path, *, apply_labels: bool, include_metadata: bool, encoding: str, sheet_name: str, suffix: str, overwrite: bool) -> Path:
    if not input_path.exists():
        raise FileNotFoundError(f"Input not found: {input_path}")

    output_dir.mkdir(parents=True, exist_ok=True)
    out_name = input_path.stem + suffix + ".xlsx"
    out_path = output_dir / out_name

    if out_path.exists() and not overwrite:
        print(f"SKIP: {out_path} already exists. Use --overwrite to replace.", file=sys.stderr)
        return out_path

    read_kwargs = {}
    if encoding:
        read_kwargs["encoding"] = encoding

    print(f"Reading {input_path} ...", file=sys.stderr)
    df, meta = pyreadstat.read_sav(str(input_path), apply_value_formats=apply_labels, formats_as_category=False, **read_kwargs)

    # Write to Excel
    print(f"Writing {out_path} ...", file=sys.stderr)
    with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
        # Data sheet
        df.to_excel(writer, index=False, sheet_name=sheet_name)

        if include_metadata:
            # Variable info
            varinfo_df = build_variable_info_df(meta)
            varinfo_df.to_excel(writer, index=False, sheet_name="Variable_Info")

            # Value labels
            labels_df = build_value_labels_df(meta.value_labels or {})
            labels_df.to_excel(writer, index=False, sheet_name="Value_Labels")

            # File-level info
            fileinfo_rows = [
                ("file_label", meta.file_label or ""),
                ("number_of_variables", len(meta.column_names or [])),
                ("number_of_rows", len(df)),
                ("weight_variable", meta.weight_variable or ""),
                ("document", (meta.document or "").strip() if meta.document else ""),
            ]
            fileinfo_df = pd.DataFrame(fileinfo_rows, columns=["key", "value"])
            fileinfo_df.to_excel(writer, index=False, sheet_name="File_Info")

    return out_path


def find_sav_files(root: Path) -> List[Path]:
    if root.is_file() and root.suffix.lower() == ".sav":
        return [root]
    # Search recursively
    return sorted(root.rglob("*.sav"))


def parse_args() -> argparse.Namespace:
    p = argparse.ArgumentParser(description="Convert SPSS .sav to Excel .xlsx")
    p.add_argument("input", help="Path to a .sav file or a directory containing .sav files")
    p.add_argument("output_dir", nargs="?", default=None, help="Optional output directory (defaults to INPUT's folder)")
    p.add_argument("--apply-labels", action="store_true", help="Replace codes by value labels where available")
    p.add_argument("--include-metadata", action="store_true", help="Add sheets with variable info and value labels")
    p.add_argument("--encoding", default="", help="Force file text encoding (rarely needed)")
    p.add_argument("--sheet-name", default="Data", help="Excel sheet name for the main data")
    p.add_argument("--suffix", default="_converted", help="Suffix appended to output filename before .xlsx")
    p.add_argument("--overwrite", action="store_true", help="Overwrite existing output files")
    return p.parse_args()


def main() -> int:
    args = parse_args()
    input_path = Path(args.input).expanduser().resolve()

    if args.output_dir:
        output_dir = Path(args.output_dir).expanduser().resolve()
    else:
        output_dir = (input_path.parent if input_path.is_file() else input_path)

    sav_files = find_sav_files(input_path)
    if not sav_files:
        print(f"No .sav files found under: {input_path}", file=sys.stderr)
        return 2

    successes = 0
    failures = 0
    outputs: List[Path] = []

    for sav in sav_files:
        try:
            out = convert_file(
                sav, output_dir,
                apply_labels=args.apply_labels,
                include_metadata=args.include_metadata,
                encoding=args.encoding,
                sheet_name=args.sheet_name,
                suffix=args.suffix,
                overwrite=args.overwrite
            )
            outputs.append(out)
            successes += 1
        except Exception as e:
            print(f"ERROR converting {sav}: {e}", file=sys.stderr)
            failures += 1

    print(f"Done. Converted: {successes}, Failed: {failures}", file=sys.stderr)
    if outputs:
        print("\nOutput files:", file=sys.stderr)
        for p in outputs:
            print(f" - {p}", file=sys.stderr)

    return 0 if failures == 0 else 1


In [ ]:
from pathlib import Path

WORKDIR = Path("/working_dir")
FOLDERNAMES = [
    "fallecidos/",
    "vehiculos involucrados/",
    "hechos y transito/",
]

APPLY_LABELS = True
INCLUDE_METADATA = True
ENCODING = ""
SHEET_NAME = "Data"
SUFFIX = "_converted"
OVERWRITE = True

# Recorre carpetas y convierte
outputs = []
errors = []

for folder in FOLDERNAMES:
    folder_path = WORKDIR / folder
    if not folder_path.exists():
        print(f"AVISO: no existe {folder_path}, se omite.")
        continue

    sav_files = find_sav_files(folder_path)
    if not sav_files:
        print(f"Sin .sav en {folder_path}")
        continue

    for sav in sav_files:
        try:
            out = convert_file(
                input_path=sav,
                output_dir=sav.parent,          # mismo directorio que el .sav
                apply_labels=APPLY_LABELS,
                include_metadata=INCLUDE_METADATA,
                encoding=ENCODING,
                sheet_name=SHEET_NAME,
                suffix=SUFFIX,
                overwrite=OVERWRITE
            )
            print(f"OK → {out}")
            outputs.append(out)
        except Exception as e:
            print(f"ERROR con {sav}: {e}")
            errors.append((sav, e))

print(f"\nResumen: {len(outputs)} convertidos, {len(errors)} con error")
